In [23]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


In [24]:
dataset_dir = 'datset' 
images = []
labels = []
label_map = {'happy': 0, 'sad': 1, 'angey': 2}

In [25]:
for label_name in os.listdir(dataset_dir):
    label_path = os.path.join(dataset_dir, label_name)
    

    if not os.path.isdir(label_path) or label_name.startswith('.'):
        continue
    
    for img_name in os.listdir(label_path):
        img_path = os.path.join(label_path, img_name)
        
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        try:

            img = load_img(img_path, target_size=(64, 64))  
            img_array = img_to_array(img) / 255.0 
            

            images.append(img_array)
            labels.append(label_map[label_name])
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            continue


In [26]:
images = np.array(images)
labels = np.array(labels)

In [27]:

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


print(f"Train images shape: {X_train.shape}")
print(f"Test images shape: {X_test.shape}")

Train images shape: (652, 64, 64, 3)
Test images shape: (164, 64, 64, 3)


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3))) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

In [29]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [40]:
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    batch_size=32, 
                    validation_data=(X_test, y_test))

Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.4400 - loss: 1.4059 - val_accuracy: 0.6402 - val_loss: 0.7263
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.7882 - loss: 0.5956 - val_accuracy: 1.0000 - val_loss: 0.2142
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9566 - loss: 0.1730 - val_accuracy: 1.0000 - val_loss: 0.0400
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.9911 - loss: 0.0589 - val_accuracy: 1.0000 - val_loss: 0.0089
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.9993 - loss: 0.0259 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.9977 - loss: 0.0179 - val_accuracy: 1.0000 - val_loss: 9.7858e-04
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.9992 - loss: 0.0060 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9880 - loss: 0.0232 - val_accuracy: 1.0000

In [42]:
model.save('emoji_classification_model.h5')

In [43]:
def predict_emoji(image_path):
 
    img = load_img(image_path, target_size=(64, 64)) 
    img_array = img_to_array(img) / 255.0 
    
    
    img_array = np.expand_dims(img_array, axis=0)  
    prediction = model.predict(img_array)  
    
    
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    
    label_map = {0: 'Happy', 1: 'Sad', 2: 'Angry'}
    predicted_label = label_map[predicted_class]
    
    return predicted_label

#'archive/image/Apple/1.png
#archive/image/Facebook/28.png
#'archive/image/Facebook/57.png'
#'archive/image/Facebook/33.png'
#'archive/image/Facebook/103.png'
image_path = 'archive/image/Facebook/103.png'
predicted_label = predict_emoji(image_path)
print(f"Predicted label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted label: Angry


In [44]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 1.9898e-04
Test Loss: 0.00021050153009127825
Test Accuracy: 100.00%
